# Especialização em Ciência de Dados - PUC-Rio
## Machine Learning - Prof. Tatiana Escovedo
### Pré-Processamento de Dados

**Referências:** https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

In [1]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Importação de pacotes
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # para normalização
from sklearn.preprocessing import StandardScaler # para padronização
from sklearn.preprocessing import OrdinalEncoder # para ordinal encoding
from sklearn.preprocessing import OneHotEncoder # para one-hot encoding e dummy encoding

## Transformações Numéricas

Muitos algoritmos de machine learning apresentam um melhor desempenho quando os atributos de entrada numéricos são redimensionados (exemplos: algoritmos que trabalham com a soma ponderada de entradas, como regressão linear, regressão logística e redes neurais; e algoritmos baseados em distância ou produto interno dos atributos de entrada, como o KNN e SVM).

As duas técnicas mais populares para redimensionar dados numéricos antes da modelagem são:

* **Normalização:** dimensiona cada atributo separadamente entre 0 e 1
* **Padronização:** dimensiona cada atributo separadamente subtraindo a média e dividindo pelo desvio padrão, transformando em uma distribuição normal padrão com média 0 e desvio padrão 1.

Já sabemos que modelos de Machine Learning (de aprendizado supervisionado) mapeiam os atributos de entrada em uma variável de saída, e que a escala e a distribuição dos dados podem ser diferentes para cada atributo do dataset, bem como terem diferentes unidades.

Estas diferenças podem aumentar a dificuldade do problema a ser modelado. Além disso, atributos de entrada com valores muito grandes podem resultar em um modelo que aprende pesos com valores altos, o que pode tornar o modelo instável e com performance de aprendizado ruim, pois pode ser muito sensível às diferenças dos valores de entrada, podendo resultar em um alto erro de generalização.

A diferença na escala dos atributos não afeta todos os algoritmos de machine learning. Geralmente, os algoritmos baseados em árvores (árvores de decisão ou random forest) não são afetados.

*Assunto de MLA: Em redes neurais, também pode ser interessante redimensionar a variável de saída de problemas de regressão, para tornar o aprendizado mais fácil (devido à questão do erro do gradiente descendente: variáveis de saída muito grandes fazem com que os pesos se modifiquem muito bruscamente, tornando o processo de aprendizado instável). Ao final do aprendizado, na aplicação do modelo, a variável de saída pode ser pós-processada, para que seja informado o valor de saída real.*

Podemos utilizar as operação de normalização e padronização usando a biblioteca **Scikit-learn**.

In [2]:
# dados que iremos usar nos exemplos
data = np.asarray([[100, 0.001],
				[8, 0.05],
				[50, 0.005],
				[88, 0.07],
				[4, 0.1]])
print(data)

[[1.0e+02 1.0e-03]
 [8.0e+00 5.0e-02]
 [5.0e+01 5.0e-03]
 [8.8e+01 7.0e-02]
 [4.0e+00 1.0e-01]]


### Normalização

A Normalização redimensiona os dados do intervalo original para um novo intervalo entre 0 e 1. São utilizados os valores mínimo e máximo observáveis, sendo possível estimar esses valores a partir dos dados disponíveis.

Notmalizamos os dados dividindo todos os valores pelo valor máximo encontrado ou subtraindo o valor mínimo e dividindo pelo intervalo entre os valores máximo e mínimo:

*y = (x – min) / (max – min)*

Podemos normalizar os dados usando o objeto **MinMaxScaler**, do pacote **Scikit-learn**. A escala padrão é o intervalo [0,1], mas é possível especificar outro intervalo através do parâmetro *feature_range*, que será utilizado para todas as variáveis normalizadas.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [3]:
# definindo o transformador como min max scaler
scaler = MinMaxScaler()

# transformando os dados
scaled = scaler.fit_transform(data)
print(scaled)

[[1.         0.        ]
 [0.04166667 0.49494949]
 [0.47916667 0.04040404]
 [0.875      0.6969697 ]
 [0.         1.        ]]


### Padronização

A Padronização redimensiona a distribuição dos valores observados para que a sua média seja 0 e o seu desvio padrão 1, também conhecida como subtração do valor médio ou centralização dos dados.

A padronização pressupõe que suas observações sigam uma distribuição Normal. Ainda é possível padronizar seus dados mesmo se essa premissa não for verdadeira, mas talvez os resultados sejam prejudicados.

Para padronizar os dados, calcula-se a média estatística e o desvio padrão dos valores dos atributos, subtrai-se a média de cada valor e divide-se o resultado pelo desvio padrão. 

A padronização exige que consigamos estimar com precisão a média e o desvio padrão dos valores observáveis. Recomenda-se estimar esses valores com base nos dados de treinamento, não no conjunto de dados inteiro.

Um valor é padronizado da seguinte maneira:

* média = soma (x) / contagem (x)
* desvio_ padrão = sqrt (soma ((x - média) ^ 2) / contagem (x))
* y = (x - média) / desvio padrão

Podemos padronizar os dados usando o objeto **StandardScaler**, do pacote **Scikit-learn**.

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [4]:
# definindo o transformador como standard scaler
scaler = StandardScaler()

# transformando os dados
scaled = scaler.fit_transform(data)
print(scaled)

[[ 1.26398112 -1.16389967]
 [-1.06174414  0.12639634]
 [ 0.         -1.05856939]
 [ 0.96062565  0.65304778]
 [-1.16286263  1.44302493]]


**Quando Normalizar e quando Padronizar?**

* Se a distribuição é normal, padronize. Caso contrário, normalize.
* Os problemas de modelagem preditiva são muitas vezes complexos, não sendo clara a melhor transformação para realizar.
* Na dúvida, use a normalização. Se tiver tempo, explore os modelos com os dados sem transformação, com a padronização e com a normalização e veja se os resultados são significativamente diferentes e se o custo x benefício vale a pena.
* Como a padronização resulta em valores positivos e negativos, pode ser interessante normalizar os dados após a padronização.
* É possível definir os valores de mínimo e máximo de acordo com o conhecimento no negócio (e não simplesmente se ater aos valores observados)

## Transformações Categóricas

Algumas implementações de modelos de Machine Learning requerem que os atributos sejam numéricos, sendo necessário codificar os atributos categóricos em numéricos antes de treinar e utilizar o modelo.

Usamos o **ordinal encoding** para as variáveis categóricas ordinais e o **one-hot encoding** para as variáveis categóricas nominais.

OBS: em alguns lugares, se usa "variáveis nominais" como sinônimo de "variáveis categóricas".

In [5]:
# dados que iremos usar nos exemplos
data = np.asarray([['red'], ['green'], ['blue']])
print(data)

[['red']
 ['green']
 ['blue']]


### Ordinal Encoding

No *ordinal encoding*, cada categoria única é transformada em um número inteiro, mantendo o relacionamento ordinal entre as variáveis. Este encoding não deve ser utilizado quando a variável não é ordinal, pois seria criada uma ordenação que não existe.

O *ordinal encoding* é implementado pelo objeto **OrdinalEncoder**, do pacote **Scikit-learn**. Por padrão, serão atribuídos inteiros aos valores categóricos ordinais em ordem alfabética, mas é possível especificar a ordenação desejada através do parâmetro *categories*.

A classe OrdinalEncoder é utilizada com atributos em formato matricial (organizadas em linhas e colunas). Se quisermos codificar uma variável target categórica, devemos usar a classe **LabelEncoder**, similar ao **OrdinalEncoder**, mas que espera um input de 1 dimensão apenas.

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [6]:
# definindo o transformador como ordinal encoding
encoder = OrdinalEncoder()

# transformando os dados
result = encoder.fit_transform(data)
print(result)

[[2.]
 [1.]
 [0.]]


### One Hot Encoding

Para variáveis categóricas nominais, sem ordenação existente entre elas, devemos utilizar o one-hot encoding. Em vez de uma variável inteira, é criada uma variável binária para cada valor único da variável. Este nome se dá porque para cada possível categoria, apenas um bit é "ativado".

O one-hot encoding é implementado pelo objeto **OneHotEncoder**, do pacote **Scikit-learn**, que ordena as categorias alfabeticamente antes de aplicar a transformação. É possível especificar a lista de categorias através do parâmetro *categories*.

Espera-se que o conjunto de treinamento contenha pelo menos um exemplo de cada categoria se as categorias não forem explicitamente definidas. Se os novos dados (conjunto de teste, por exemplo) tiverem categorias não vistas no treinamento, é possível configurar o parâmetro "handle_unknown" como "ignore" para que não ocorra um erro.

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [7]:
# definindo o transformador como one hot encoding
encoder = OneHotEncoder(sparse=False)

# transformando os dados
onehot = encoder.fit_transform(data)
print(onehot)

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


### Dummy Variable Encoding

O one-hot encoding cria uma variável binária para cada categoria, mas esta representação inclui redundância. Por exemplo, se soubermos que [1, 0, 0] representa "azul" e [0, 1, 0] representa "verde", não precisamos de outra variável binária [0, 0, 1] para representar "vermelho". Poderíamos usar [0, 0] para "vermelho", [1, 0] para "azul" e [0, 1] para "verde". A codificação de variável dummy representa C categorias em C-1 variáveis ​​binárias.

> **Nota de nível avançado:** Em modelos de regresão (que têm termo de viés - bias) é necesário optar pelo dummy encoding no lugar do one-hot encoding, pois o one-hot encoding fará com que a matriz de dados de entrada seja singular (não-inversível), impossibilitando o cálculo dos coeficientes de regressão (que usa operações de álgebra linear). Saiba mais em: https://towardsdatascience.com/one-hot-encoding-multicollinearity-and-the-dummy-variable-trap-b5840be3c41a e https://inmachineswetrust.com/posts/drop-first-columns/

A codificação dummy também é implementada pelo objeto **OneHotEncoder**, do pacote **Scikit-learn**, usando o parâmetro *drop* para indicar qual categoria receberá todos os valores zero, chamada de "linha de base". Podemos usar *first* para que a primeira categoria seja usada (as categorias são ordenadas alfabeticamente).

In [8]:
# definindo o transformador como one hot encoding (com Dummy variable encoder)
encoder = OneHotEncoder(drop='first', sparse=False)

# transformando os dados
onehot = encoder.fit_transform(data)
print(onehot)

[[0. 1.]
 [1. 0.]
 [0. 0.]]


Se tivermos no mesmo datsset atributos dos dois tipos (numéricos e categóricos), será necessário transformar/codificar cada atributo (coluna) separadamente e concatenar todas as variáveis ​​preparadas novamente em uma única matriz para ajustar ou avaliar o modelo (ou usar o **ColumnTransformer** para aplicar condicionalmente diferentes transformações de dados a diferentes variáveis ​​de entrada).

https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

## Exercícios

In [9]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Importação de pacotes
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler # para normalização
from sklearn.preprocessing import StandardScaler # para padronização

In [10]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Informa o cabeçalho das colunas
colunas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=0, delimiter=',')

In [11]:
# Pegando apenas os dados do dataset e guardando em um array
array = dataset.values

# Separando o array em componentes de input e output
X = array[:,0:8]
Y = array[:,8]

In [ ]:
# 1. Normalize o dataset usando MinMaxScaler

In [ ]:
# 2. Padronize o dataset usando StandardScaler 

## Gabarito

In [ ]:
# 1. Normalize o dataset usando MinMaxScaler

# Normalizando os dados
scaler = MinMaxScaler().fit(X)
normalizedX = scaler.transform(X)

# Sumarizando os dados transformados
print("Dados Originais: \n\n", dataset.values)
print("\nDados Normalizados: \n\n", normalizedX[0:5,:])

Dados Originais: 

 [[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]

Dados Normalizados: 

 [[0.35294118 0.74371859 0.59016393 0.35353535 0.         0.50074516
  0.23441503 0.48333333]
 [0.05882353 0.42713568 0.54098361 0.29292929 0.         0.39642325
  0.11656704 0.16666667]
 [0.47058824 0.91959799 0.52459016 0.         0.         0.34724292
  0.25362938 0.18333333]
 [0.05882353 0.44723618 0.54098361 0.23232323 0.11111111 0.41877794
  0.03800171 0.        ]
 [0.         0.68844221 0.32786885 0.35353535 0.19858156 0.64232489
  0.94363792 0.2       ]]


In [ ]:
# 2. Padronize o dataset usando StandardScaler 

# Padronizando os dados
scaler = StandardScaler().fit(X)
standardX = scaler.transform(X)

# Sumarizando os dados transformados
print("Dados Originais: \n\n", dataset.values)
print("\nDados Padronizados: \n\n", standardX[0:5,:])

Dados Originais: 

 [[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  1.     85.     66.    ...   0.351  31.      0.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 ...
 [  5.    121.     72.    ...   0.245  30.      0.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]
 [  1.     93.     70.    ...   0.315  23.      0.   ]]

Dados Padronizados: 

 [[ 0.63994726  0.84832379  0.14964075  0.90726993 -0.69289057  0.20401277
   0.46849198  1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575  0.53090156 -0.69289057 -0.68442195
  -0.36506078 -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 -1.28821221 -0.69289057 -1.10325546
   0.60439732 -0.10558415]
 [-0.84488505 -0.99820778 -0.16054575  0.15453319  0.12330164 -0.49404308
  -0.92076261 -1.04154944]
 [-1.14185152  0.5040552  -1.50468724  0.90726993  0.76583594  1.4097456
   5.4849091  -0.0204964 ]]
